# Codage de Huffman et compression de texte

Le codage de Huffman, inventé par David Albert Huffman, et publié en 1952, est un algorithme de compression de données sans perte. Le codage de Huffman utilise un code à longueur variable pour représenter un symbole de la source (par exemple un caractère dans un fichier). Le code est déterminé à partir d'une estimation des probabilités d'apparition des symboles de source, un code court étant associé aux symboles de source les plus fréquents.

**Le principe du codage de Huffman repose sur la création d'une structure d'arbre composée de nœuds**. L'arbre est créé de la manière suivante, on associe chaque fois les deux nœuds (2 feuilles ou 1 feuilles avec un noeud) de plus faibles poids, pour donner un nouveau nœud dont le poids équivaut à la somme des poids de ses fils. On réitère ce processus jusqu'à n'en avoir plus qu'un seul nœud : la racine. On associe ensuite par exemple le code 0 à chaque embranchement partant vers la gauche et le code 1 vers la droite.

__Exemple__ :
Pour une séquence contenant les caractères (a, b, c, d, e) d'occurences respectives (1, 1, 2, 2, 3) on obtient le graphe suivant :
![Huffman Example](huffman_1.png "Exemple")

Ce qui donne donc le codage de Huffman suivant :

| Symbole | Fréquence | Code |
|---------|-----------|------|
| a       |         1 |  100 |
| b       |         1 |  101 |
| c       |         2 |   01 |
| d       |         2 |   00 |
| e       |         3 |   11 |
### Objectifs 

1. Nettoyer un texte pour ne garder que les lettres de l’alphabet.  
2. Calculer l'entropie du texte et en déduire le minimum nombre de bits/signes nécessaire pour un encodage efficace.
3. Construire l'arbre de Huffman.
4. Coder et Décoder (Huffman)
5. Visualiser l'arbre de Huffman et exporter l'image (En option).
6. Calculer les taux de compression et les temps de calcul pour un paragraphe de test. 
7. Vérifier le décodage
8. Comparer le codage binaire classique (mot de code fixe) et le codage de Huffman.
9. Compression de fichiers

## 1. Nettoyage du texte

In [12]:
def clean_message(message: str) -> str:
    message = message.lower()
    # isalpha(): garde les lettres
    # ici on supprime tous les caractères qui ne sont pas lettre/espace
    return "".join([char for char in message if char.isalpha() or char == ' '])

texte = "Il n'existe que deux choses infinies : l'univers et la bêtise humaine !"
texte_nettoye = clean_message(texte)
print("Texte nettoyé :", texte_nettoye)

Texte nettoyé : il nexiste que deux choses infinies  lunivers et la bêtise humaine 


## 2. Calcul de l'entropie
**L'entropie de Shannon** est donnée par : 
\begin{align*}
    H(S) = - \sum_{i=1}^{N} p_i\, \log_{2}\left(p_i\right)
\end{align*}
où $p_i$ est la probabilité d'apparation du signe $i$.

C'est la quantité moyenne d'information par caractère:
- haute => caractères imprévisibles (donc apporte beaucoup d'information).
- basse => caractères redondants (donc apporte peu d'information).

In [13]:
import math

def entropie(message: str) -> float:
    # message vide => aucune information
    if not message:
        return 0.0
    freq = {}
    for c in message:
        freq[c] = freq.get(c,0)+1
    total = len(message)
    probs = [f/total for f in freq.values()]
    return -sum(p*math.log2(p) for p in probs)

H = entropie(texte_nettoye)
print(f"Entropie du texte : {H:.3f} bits/symbole")

Entropie du texte : 3.814 bits/symbole


## 3. Construction de l'arbre de Huffman

In [14]:
class HuffmanNode:
    def __init__(self, freq, data, left=None, right=None):
        self.freq = freq
        self.data = data
        self.left = left
        self.right = right

def generate_tree(freq_map):
    # créé un noeud initial par caractère
    nodes = [HuffmanNode(freq, char) for char, freq in freq_map.items()]
    # trie du plus petit au plus grand 
    nodes = sorted(nodes, key=lambda x: x.freq)
    # à chaque tour:
    while len(nodes) > 1:
        # prend les deux noeuds les moins fréquents
        # les fusionne dans nouveau noeud dont la freq est la somme
        # on remet le noeud dans la liste
        left = nodes.pop(0)
        right = nodes.pop(0)
        merged = HuffmanNode(left.freq + right.freq, '-', left, right)
        nodes.append(merged)
        nodes = sorted(nodes, key=lambda x: x.freq)
    # à la fin, on obtient la racine de l'arbre de huffman
    return nodes[0]

def set_binary_code(node, prefix, mapping):
    # si le noeud est une feuille (donc un caractère), on lui associe
    # le code binaire doncstruit dans prefix
    if node.left is None and node.right is None:
        mapping[node.data] = prefix
        return
    if node.left:
        set_binary_code(node.left, prefix + '0', mapping)
    if node.right:
        set_binary_code(node.right, prefix + '1', mapping)

def huffman_encode(message: str):
    freq_map = {}
    # compte la fréquence de chaque caractère
    for c in message:
        freq_map[c] = freq_map.get(c, 0) + 1
    # génère l'arbre et les codes binaires
    root = generate_tree(freq_map)
    mapping = {}
    set_binary_code(root, '', mapping)
    # converti le message entier en binaire en remplaçant chaque 
    # caractère par son code Huffman
    code = ''.join(mapping[c] for c in message)
    # code: le message compressé en bits
    # mapping: le dictionnaire {caractère, code}
    # root: l'arbre (pour décoder)
    return code, mapping, root

def huffman_decode(code: str, root: HuffmanNode) -> str:
    message = ''
    # lit chaque bit et descend deans l'arbre
    node = root
    for bit in code:
        node = node.left if bit == '0' else node.right
        # on atteint une feuille 
        if node.left is None and node.right is None:
            # récupère le caractère
            message += node.data
            # repart à la racine pour lire le reste
            node = root
    return message

In [15]:
code_huff, mapping, root = huffman_encode(texte_nettoye)
print(f"Longueur totale Huffman: {len(code_huff)} bits")

Longueur totale Huffman : 258 bits


## 4. Codage et décodage Huffman

In [18]:
def huffman_encode(message: str):
    freq_map = {}
    for x in message:
        if x in freq_map:
            freq_map[x] += 1
        else:
            freq_map[x] = 1
    arbre = generate_tree(freq_map)
    code_mapping = {}
    set_binary_code(arbre, "", code_mapping)
    ch = ""
    for x in message:
        ch += code_mapping[x]
    return ch, code_mapping, arbre

def huffman_decode(code: str, root: HuffmanNode) -> str:
    mapping = {}
    set_binary_code(root, "", mapping)
    
    def find_current_matches(mapping, current_code):
        l = []
        for k,v in mapping.items():
            if v.startswith(current_code):
                l.append(k)
        return l
    
    current_code = ""
    message_decode = ""
    i = 0
    while i < len(code):
        matches = find_current_matches(mapping, current_code)
        while len(matches) > 1:
            current_code += code[i]
            i += 1
            matches = find_current_matches(mapping, current_code)
        message_decode += matches[0]
        current_code = ""
    return message_decode
        
    
code_huff, mapping, root = huffman_encode(texte_nettoye)
print(f"Longueur totale Huffman: {len(code_huff)} bits")

Longueur totale Huffman: 258 bits


## 5. Visualisation de l'arbre de Huffman

In [20]:

from graphviz import Digraph

def export_huffman_tree(node, filename="huffman_tree.png"):
    dot = Digraph(format="png")
    dot.attr("node", shape="circle", fontname="Helvetica")

    def add_nodes_edges(node, parent=None):
        if node is None:
            return

        if hasattr(node, "char") and node.char is not None:
            label = f"{node.char}\\n{node.freq}"
        else:
            label = str(node.freq)

        dot.node(str(id(node)), label)
        if parent is not None:
            dot.edge(str(id(parent)), str(id(node)))

        if hasattr(node, "left"):
            add_nodes_edges(node.left, node)
        if hasattr(node, "right"):
            add_nodes_edges(node.right, node)

    add_nodes_edges(node)
    dot.render(filename, cleanup=True)

export_huffman_tree(root)
print("Arbre Huffman exporté en huffman_tree.png")

Arbre Huffman exporté en huffman_tree.png


## 6. Calcul du taux de compression
En se basant sur le codage binaire brut, que peut-on dire du rôle du codage de Huffman ? Est-il plus ou moins efficace que le codage de Shannon-Fano ? Vous calculerez le taux de compression pour illustrer votre réponse.

Pour rappel : 
- Le codage de Shannon-Fano du même message est : ""
- Le taux de compression se calcul ainsi : 
\begin{align*}
\text{taux de compression} = 1 - \frac{\text{taille Huffman}}{\text{taille brute}}
\end{align*}

In [21]:
code_brut = ''.join(format(ord(c), '08b') for c in texte_nettoye)
print("longueur code brut: ", len(code_brut))
print("longueur code huff: ", len(code_huff))

def taux_compression(code_brut: str, code_huff: str) -> float:
    return 1 - (len(code_huff) / len(code_brut))

tc = taux_compression(code_brut, code_huff)
print(f"Taux de compression: {tc*100:.2f}%")

longueur code brut:  536
longueur code huff:  258
Taux de compression: 51.87%


Pour le code brut on se base ici sur le fait que la représentation binaire d'un caractère ascii utilise généralement 8 bits.

## 7. Vérification du décodage
Vous allez vérifier que si on décode le message codé, on retrouve le message intiial !

In [22]:
decoded = huffman_decode(code_huff, root)
print(f"Décodage correct ? {decoded == texte_nettoye}")

Décodage correct ? True


## 8. Codage binaire classique
Dans le codage binaire brut (ASCII simplifié), on attribue un même nombre de bits à chaque caractère.
- Si le texte contient $n$ signes distincts, il faut au moins : $\lceil \log_2\left(n\right)\rceil$ bits par symbole.

In [23]:
def codage_binaire_brut(message: str) -> tuple[str, int]:
    symboles = sorted(set(message))
    n = len(symboles)
    bits_par_symbole = math.ceil(math.log2(n)) if n > 0 else 0
    codes = {sym: format(i, f'0{bits_par_symbole}b') for i, sym in enumerate(symboles)}
    code_brut = ''.join(codes[c] for c in message)
    return code_brut, bits_par_symbole

code_brut, bits_brut = codage_binaire_brut(texte_nettoye)
print(f"Codage brut : {bits_brut} bits/symbole, longueur totale = {len(code_brut)} bits")

Codage brut : 5 bits/symbole, longueur totale = 335 bits


## 9. Taux de compression & Temps de calcul
Vous allez maintenant pouvoir faire une petite comparaison entre les deux encodages (Huffman et Binaire), en terme de :

1. Temps de calcul
2. Taux de compression

Pour un texte conséquent (le texte contenu dans le fichier `paragraph_test.txt`)


In [24]:
from time import perf_counter
def timer():
    def wrapper(func):
        start = perf_counter()

In [25]:
def benchmark():
    text = "Le codage de Huffman est un algorithme de compression de données sans perte. Le codage de Huffman\
    utilise un code à longueur variable pour représenter un symbole de la source (par exemple un caractère dans\
    un fichier). Le code est déterminé à partir d'une estimation des probabilités d'apparition des symboles de\
    source, un code court étant associé aux symboles de source les plus fréquents.Un code de Huffman est optimal\
    au sens de la plus courte longueur pour un codage par symbole, et une distribution de probabilité connue.\
    Des méthodes plus complexes réalisant une modélisation probabiliste de la source permettent d'obtenir de\
    meilleurs ratios de compression. Il a été inventé par David Albert Huffman, et publié en 1952."
    
    text_clean = clean_message(text)
    code_brut = ''.join(format(ord(c), '08b') for c in text_clean)
    
    start = perf_counter()
    code_huffman, _, _ = huffman_encode(text_clean)
    end = perf_counter()
    
    print(f"Codage de huffman\nTemps : {(end - start)*1000:.2f}ms\nCompression : {taux_compression(code_brut, code_huffman)*100:.2f}%")
    
    start = perf_counter()
    code_binaire = codage_binaire_brut(text_clean)
    end = perf_counter()
    
    print(f"\nCodage binaire brut\nTemps : {(end - start)*1000:.2f}ms\nCompression : {taux_compression(code_brut, code_binaire)*100:.2f}%")
benchmark()

Codage de huffman
Temps : 0.99ms
Compression : 48.57%

Codage binaire brut
Temps : 0.35ms
Compression : 99.97%


Le codage de Huffman offre une compression efficace d’environ 48 %, au prix d’un temps de calcul légèrement supérieur. Le codage binaire brut, bien plus rapide, ne fournit pratiquement aucune compression, conservant quasiment la même taille que le message original.

## 10. Compression décompression de fichier

Nous allons essayer ici de compresser des fichiers textes conséquents à l'aide de l'encodage de Huffman. Pour ce faire, nous devons : 

1. Ecrire la fonction `decode_from_dico`
2. Ecrire les fonctions `compress_file` & `decompress_file`

Le fichier compressé doit contenir : 

+ Le code convertit en bytes (avec un padding de '0' pour avoir une taille multiple de 8).
+ Le dictionnaire d'encodage et le padding

Utiliser le module `pickle` de python, pour vous faciliter la tâche !


In [26]:
import pickle
def decode_from_dico(dico: dict, code: str) -> str:
    rev_dico: dict = {v:k for k, v in dico.items()}
    msg = ''
    current_code = ''
    for bit in code:
        current_code += bit
        if current_code in rev_dico:
            msg += rev_dico[current_code]
            current_code = ''
    return msg

def compress_file(input_path: str, output_path: str) -> None:
    with open(input_path, 'rb') as f:
            data = f.read()
        
    # Conversion en string pour l'encodage
    text = data.decode('utf-8', errors='ignore')
    
    # Encodage Huffman
    
    encoded, codes, _ = huffman_encode(text)
    #print(f'{encoded = }, {codes = }')
    # Conversion du string binaire en bytes
    padding = 8 - len(encoded) % 8
    #print(f'{padding = }')
    encoded_padded = encoded + "0" * padding
    #print(f'{len(encoded) = }, {len(encoded_padded) = }')
    bytes_array = bytearray()
    for i in range(0, len(encoded_padded), 8):
        byte = encoded_padded[i:i+8]
        #print(f'{byte = }, {int(byte, 2) = }')
        bytes_array.append(int(byte, 2))
    
    # Sauvegarde        
    with open(output_path, 'wb') as f:
        pickle.dump((bytes(bytes_array), codes, padding), f)
    
    # Calcul des statistiques
    size_original = len(data)
    size_compressed = len(bytes_array)
    ratio = (1 - size_compressed / size_original) * 100
    
    print(f"✓ Compression réussie!")
    print(f"  Taille originale: {size_original:,} bytes")
    print(f"  Taille compressée: {size_compressed:,} bytes")
    print(f"  Ratio: {ratio:.2f}% de réduction")

def decompress_file(input_path, output_path):
    # Chargement du fichier compressé
    with open(input_path, 'rb') as f:
        bytes_data, codes, padding = pickle.load(f)
        
    # Conversion des bytes en string binaire
    encoded_padded = "".join(format(byte, '08b') for byte in bytes_data)
    
    # Suppression du padding
    encoded = encoded_padded[:-padding] if padding else encoded_padded
    
    # Décodage Huffman
    
    text = decode_from_dico(codes, encoded)
    
    # Sauvegarde
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(text)
    
    print(f"✓ Décompression réussie!")
    print(f"  Fichier extrait: {output_path}")

In [28]:
test_file = 'bigfile.txt'
comp_file = f"{test_file.removesuffix('.txt')}_comp.bin"
decomp_file = f"{test_file.removesuffix('.txt')}_decomp.txt"
compress_file(test_file, comp_file)
decompress_file(comp_file, decomp_file)

# Vérification
with open(test_file, 'r') as f:
        original = f.read()
with open(decomp_file, 'r') as f:
        decompressed = f.read()
    
print("\n" + "=" * 50)
print("VÉRIFICATION")
print("=" * 50)
print(f"Fichiers identiques: {original == decompressed}")

✓ Compression réussie!
  Taille originale: 3,971 bytes
  Taille compressée: 2,167 bytes
  Ratio: 45.43% de réduction
✓ Décompression réussie!
  Fichier extrait: bigfile_decomp.txt

VÉRIFICATION
Fichiers identiques: True
